In [117]:
import chardet
import pandas as pd
import re
from time import strptime

In [15]:
fpath = 'subtitles/Westworld - 2x01 - Journey Into Night.WEB.DEFLATE.en.srt'
with open(fpath, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [128]:
subtitle_dict = {
    'subtitle_index': [],
    'start': [],
    'end': [],
    'text': []
}

fpath = 'subtitles/Westworld - 2x01 - Journey Into Night.WEB.DEFLATE.en.srt'
with open(fpath, 'r', encoding='ISO-8859-1') as f:
    text_accumulator = []  # gathers multi-line subtitles
    for line in f:
        index_match = re.match(r'([0-9]+)\n', line)
        newline_match = re.fullmatch(r'\n', line)
        
        if index_match:
            # line is subtitle index
            index_int = int(index_match.group(1))
            subtitle_dict['subtitle_index'].append(index_int)
        elif re.search(r'-->', line):
            try:
                start, end = re.findall(r'([0-9]{2}:[0-9]{2}:[0-9]{1,2},[0-9]{2,3})', line)
            except ValueError:
                print(line)
            #subtitle_dict['start'].append(strptime(start, '%H:%M:%S,%f'))
            #subtitle_dict['end'].append(strptime(end, '%H:%M:%S,%f'))
            subtitle_dict['start'].append(start)
            subtitle_dict['end'].append(end)
        elif newline_match:
            # blank line between subtitles
            full_subtitle = ' '.join(text_accumulator)
            subtitle_dict['text'].append(full_subtitle)
            text_accumulator = []  # reset accumulator
        else:
            # text of the dialogue
            text_accumulator.append(line.strip())
    # Save the text contents of the last loop
    full_subtitle = ' '.join(text_accumulator)
    subtitle_dict['text'].append(full_subtitle)
            
for k, v in subtitle_dict.items():
    print(k, len(v))

subtitle_index 701
start 701
end 701
text 701


In [102]:
all_lines[6].strip()

'Welcome to Westworld.'

In [129]:
df = pd.DataFrame.from_dict(subtitle_dict)
df.head()

,subtitle_index,start,end,text
0,1,"00:00:00,018","00:00:01,251",_
1,2,"00:00:01,254","00:00:02,802",Welcome to Westworld.
2,3,"00:00:02,805","00:00:04,262",Which would you prefer?
3,4,"00:00:05,262","00:00:07,303",This place is the answer
4,5,"00:00:07,304","00:00:09,590",to that question that you've been asking yours...


In [133]:
for col in ['start', 'end']:
    df[col] = pd.to_datetime(df[col], format='%H:%M:%S,%f').dt.time

In [137]:
# Drop font color rows
df = df.drop(index=df.loc[df.text.str.contains('<font')].index)

In [138]:
df

,subtitle_index,start,end,text
0,1,00:00:00.018000,00:00:01.251000,_
1,2,00:00:01.254000,00:00:02.802000,Welcome to Westworld.
2,3,00:00:02.805000,00:00:04.262000,Which would you prefer?
3,4,00:00:05.262000,00:00:07.303000,This place is the answer
4,5,00:00:07.304000,00:00:09.590000,to that question that you've been asking yours...
...,...,...,...,...
694,695,01:11:33.979000,01:11:36.354000,which means I need yours.
695,696,01:11:38.081000,01:11:40.089000,Can you tell me what happened?
696,697,01:11:58.730000,01:12:01.831000,I...
697,698,01:12:01.834000,01:12:03.831000,I killed them.
